# ALI numerical algorithm for two-stream transfer

## Imports

In [ ]:
import numpy as np

## 1) Set parameters and boundary values

### Set the grid

In [ ]:
# Set the number of grid points
n = 5

# Set the 1D grid
z = np.linspace(0.,1.,num=n)

# Set delta z (grid spacing value)
del_z = z[1]

### Gas variables

In [ ]:
# Set the thermal source function (aka the Planck function)
# In this example it is constant at each depth
B = np.ones(n)

# Photon destruction probability
# In this example it is constant 
epsilon = np.ones(n)*0.1

In [ ]:
# Set the opacities
# They will all be a function of height
# Or constant
def alpha(z):
    """
    Input: Depth z 
    Output: Total opacity at the depth value given
    """
    
    return(10.**(5. - 6.*z)) # This is actually the extinction coefficient

# For the example problem this is 0
# but it will not always be 0
def alpha_abs(z):
    """
    Input: Depth z 
    Output: absorption opacity at the depth value given
    """
    
    return(0.)

# For the example problem this is 
# the same as the total opacity
# but not always
def alpha_scat(z):
    """
    Input: Depth z 
    Output: Scattering opacity at the depth value given
    """
    
    return(10.**(5. - 6.*z))

In [ ]:
alpha_t = alpha(z)
alpha_a = np.zeros((n))
alpha_s = alpha_scat(z)

In [ ]:
# Optical depth, (delta tau)
# Right now it's set based on example 4.4.5
# but there will be more options to 
# calculate this depending on what 
# information is given in the problem
# It is also constant here because our
# grid spacing is equal
tau = np.sqrt(3.)*alpha(del_z)

### Quadrature Weights

In [ ]:
# M1 Closure scheme quadrature weights
# for J, H, and K 
# aka the moments of specific intensity
omega = 2./n
mu = 1./3.

### Set boundary and initial values

In [ ]:
# Set a null grid for I, except at
# the top and bottom boundaries 
I = np.zeros((n))
I[0] = 1.

# Set J based on the M1 closure weights
J = np.zeros((n))
for i in range(0,n-1):
    J[i] = I[i]*omega

# Set the initial pressure tensor
# based on the M1 closure weights
K = np.zeros((n,n))
for i in range(0,n-1):
    K[i,i] = I[i]*omega*mu

## 2) Create interpolation functions and a matrix solver

### Quadratic Bezier Interpolation for I(U)

In [ ]:
# Interpolation coefficients 
# Only need interpolation if working in 2D or 3D
# Still not exactly sure how to define t
#t = 0.5
#m = 1. - t

#def I_interp(i,j):
#    """
#    Input: location on grid
#        i = x-coordinate 
#        j = y-coordinate
#    Output: Value of thrid-order-quadrature specific intensity value 
#        at the input location on grid
#    """
#    return(I[i-1,j]*u**2 + I[i+1,j]*2.*u*t + I[i+2,j]*t**2)

### Third order quadratic interpolation for the source contribution

In [ ]:
# Interpolation coefficients
e0 = 1. - np.exp(-tau)
e1 = tau - e0
e2 = tau**2 - 2.*e1
u = e0 + (e2 - 3.*tau*e1)/(2.*tau**2)
p = ((2.*tau*e1)-e2)/(tau**2)
d = (e2 - tau*e1)/(2.*tau**2)

# Define the initial source function
S = epsilon*B + (np.ones(n) - epsilon)*J

# Define the 1D source term interpolation function
def S_interp(i):
    """
    Input: location on grid
    Output: Value of thrid-order-quadrature source value 
        at the input location on grid
    """
    return(u*S[i+1] + p*S[i] + d*S[i-1])

### Partial Lambda Operator

In [ ]:
# These are the lower diagonal,
# diagonal, and upper diagonal
# pieces of the simplified lambda
# operator. These will vary across 
# the grid if tau varies with z

# Lower Diagonal 
lmbda_l = 0.5*(u+w)

# Diagonal 
lmbda_d = 0.5*(v+v)

# Upper Diagonal
lmbda_u = 0.5*(w+u)

# Construct the operator
lmbda = np.zeros(n,n)
for i in range(0,n-1):
    lmbda[i,i] = lmbda_d
    if i==0:
        lmbda[i,i+1] = lmbda_u
    elif i==(n-1):
        lmbda[i,i-1] = lmbda_l
    else:
        lmbda[i,i+1] = lmbda_u
        lmbda[i,i-1] = lmbda_l

## M matrix

In [ ]:
# M* = [Identity - (1-epsilon)*Lambda*]
# Where Lambda* is the partial lambda operator 
# The matrix equation that will be solved is:
# M*S = epsilon*B
M = np.identity(n) - (np.ones(n)-epsilon)*lmbda

### Forward elimination and backward substitution subroutine

In [ ]:
# Define the functions for the coefficients
# used in the Thomas Algorithm which is used 
# and explained below
def gamma_beta_fn(mat, y, n):
    """
    Input:
        M = the tridiagonal matrix to be sovled (nxn)
        y = the solution array in the matrix equation: Mx = y (nx1)
        n = the number of rows in matrix m
    Output:
        g = the array of gamma constants (nx1)
        b = the array of beta constants (nx1)
    """
    
    # Define column arrays for necessary row-specific constants
    # This automatically sets gamma[0] = beta[0] = 0 which is 
    # necessary for the problem
    g = np.zeros((n))
    b = np.zeros((n))
    
    for i in range(0,n-2):
        if i==0:
            g[i+1] = ((-mat[i,i+1])/mat[i,i])
            b[i+1] = y[i]/mat[i,i]
        else:
            g[i+1] = ((-mat[i,i+1])/(g[i]*mat[i,i-1] + mat[i,i])) 
            b[i+1] = (y[i] - b[i]*mat[i,i-1])/mat[i,i]
    
    return(g,b)

In [ ]:
# Thomas Algorithm
# Forward elimination for a tridiagonal matrix
# Based on the algorithm found in Dullemond Ch4 notes
# Backward Substition is then applied
def tri_solver(M,S,epsilon,B,del_z):
    """
    Input: 
        J = specific intensity at every grid point
        alpha_t = total opacity
        alpha_a = absorption opacity
        B = thermal source function
    Output:
        A new matrix or vector of J values 
    """
    
    #-----------------------------------------------------------
    # Perform forward elimination and backward substitution on M
    # Thomas Algorithm: a simplified form of Gaussian Elimination
    # specifically for tridiagonal matrices 
    
    # Calculate Y = alpha_a(J - B) from MJ = alpha_a(J - B) = Y
    Y = epsilon*B
    # Calculate the necessary gamma and beta constants
    # This is essentially storing information obtained from 
    # forward substitution 
    gamma, beta = gamma_beta_fn(M, Y, n)
    # We use these constants to complete the calculation for 
    # what is essentially a back substitution to finish the
    # calculation of MX = Y
    X = np.zeros((n+1))
    X[0:n] = S
    for k in range(1,n):
        # Actually need to go from n, n-1, n-2,...,1
        # So create a new place holder that goes backwards
        c = (n) - k
        X[c-1] = gamma[c]*X[c] + beta[c]
    
    # Return the new solution to J
    return(X[0:n])

## 3) Solve for converged S

In [ ]:
# Solve for S -- I don't think this will be here...
# ----------------------------------------------------------------------
# Solve for the source function via a formal solution
S = epsilon*B + (np.ones(n) - epsilon)*J

# Solve for I
# ----------------------------------------------------------------------
# For 2D and 3D use the function defined for 
# quadratic Bezier interpolation to find the new value for I(U)
# For 1D just use the i-1 grid point for the exponential term
# Use a different third-order-quadrature method for S contributions
for i in range(0,n-1):
    I[i] = np.exp(-tau)*I[i-1] + S_interp(i)

# Solve for J
# ----------------------------------------------------------------------
# The 1D diffusion equation is a 2nd order PDE where J is the solution
# The numerical representation used for this PDE is Central Space
# Forward substitution + backward substitution are used to solve the PDE